In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, Dense, Embedding, Dropout, Flatten
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [2]:
real = pd.read_csv('./data/True.csv')
fake = pd.read_csv('./data/Fake.csv')

In [3]:
real['type'] = 1

In [4]:
fake['type'] = 0

In [5]:
all_data = pd.concat([real, fake])

In [6]:
print(len(all_data))
print(len(all_data[all_data.date.str.contains('\w+ \d{1,}, \d{4}') | all_data.date.str.contains('\d{2}-\w{3}-\d{2}') ]))


44898
44888


## All Subjects are exclusive to either Fake news or Real news no intersection

In [7]:
all_data[(all_data['subject'] == 'Government News') & (all_data['type']==True)].head()

,title,text,subject,date,type


In [8]:
all_data = all_data.drop(['title','subject'],axis=1)

## Sort data in terms in ascending date

In [9]:
## the date columns occasionally have random values such as a article url or the article title so we are just enforcing that the date is normal and parseable through some regex
all_data =  all_data[all_data.date.str.contains('\w+ \d{1,}, \d{4}') | all_data.date.str.contains('\d{2}-\w{3}-\d{2}') ]

# Converting the date into datetime so it can be easily sorted
all_data['date']= pd.to_datetime(all_data['date'])
all_data.sort_values(by = 'date', inplace = True)


In [10]:
## First and last date
print(all_data.iloc[0].date)
print(all_data.iloc[-1].date)
print(len(all_data))
print(len(all_data[all_data.type == True]))
print(len(all_data[all_data.type == False]))

2015-03-31 00:00:00
2018-02-19 00:00:00
44888
21417
23471


### Here we are removing stop words from the text column of the rows we are removing them to remove aribitrary common words that occur in human language that dont really provide any value to the model this is also a good tradeoff to cut down on training time, Depending on intial results we may want to keep the stopwords in or edit the list of stopwords, this would be a case where the stopwords are consistently providing context to the articles and we dont want to remove them.

In [11]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
stopW = stopwords.words('english')
all_data['text'] = all_data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopW)]))

### Here we are doing a technique called lemmitization which basically maps words that have the same meaning to a single word (the root form found in a Lemma) that still conveys same meaning. 
Examples
- Phone, Cell, SmartPhone -> Phone
- Hungry, starved -> Famished 


In [13]:
nltk
lemmatizer = WordNetLemmatizer()
all_data['lemmatized'] = all_data['text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

In [14]:
y = all_data.type.tolist()
corpus = all_data['lemmatized'].tolist()

In [15]:
tf.config.list_physical_devices('GPU')
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
tf.config.set_visible_devices(gpus[0], 'GPU') # unhide potentially hidden GPU
tf.config.get_visible_devices()


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [16]:
voc_size = 10000
vector_length = 5000
onehot_repr = [one_hot(words,voc_size) for words in corpus]
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=vector_length)
print(embedded_docs.shape)

(44888, 5000)


In [60]:
## Og model 
#Creating the lstm model
embedding_vector_features=64
es = EarlyStopping(monitor='val_loss',mode='min')
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=vector_length))
model.add(Dropout(0.6))
model.add(LSTM(100,return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(100))
model.add(Dropout(0.6))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

#Compiling the model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 5000, 64)          640000    
                                                                 
 dropout_3 (Dropout)         (None, 5000, 64)          0         
                                                                 
 lstm_2 (LSTM)               (None, 5000, 100)         66000     
                                                                 
 dropout_4 (Dropout)         (None, 5000, 100)         0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               80400     
                                                                 
 dropout_5 (Dropout)         (None, 100)               0         
                                                                 
 flatten (Flatten)           (None, 100)              

In [128]:
#Creating the lstm model
embedding_vector_features=24
es = EarlyStopping(monitor='val_loss',mode='min')
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=vector_length))
model.add(Dropout(0.8))
model.add(LSTM(10,return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(10))
model.add(Dropout(0.8))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

#Compiling the model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 5000, 24)          240000    
                                                                 
 dropout_24 (Dropout)        (None, 5000, 24)          0         
                                                                 
 lstm_16 (LSTM)              (None, 5000, 10)          1400      
                                                                 
 dropout_25 (Dropout)        (None, 5000, 10)          0         
                                                                 
 lstm_17 (LSTM)              (None, 10)                840       
                                                                 
 dropout_26 (Dropout)        (None, 10)                0         
                                                                 
 flatten_7 (Flatten)         (None, 10)               

## Rougly 2 years worth of data and 31437 data points and a near even split of fake and real news

In [129]:
batch_size = 256
# Converting the X and y as array
X_final=np.array(embedded_docs)
y_final=np.array(y)

#Check shape of X and y final
X_final.shape,y_final.shape

# Train test split of the X and y final
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.20, random_state=42)
Y_validation = y_test[0:1000]
X_validation = X_test[0:1000]
y_test = y_test[1000:]
X_test = X_test[1000:]
# Fitting with 10 epochs and 64 batch size
model.fit(X_train,y_train,validation_data=(X_validation,Y_validation),epochs=10,batch_size=batch_size,verbose=1,callbacks=[es])

Epoch 1/10
141/141 [==============================] - 112s 776ms/step - loss: 0.4575 - accuracy: 0.7514 - val_loss: 0.1050 - val_accuracy: 0.9670
Epoch 2/10
141/141 [==============================] - 109s 770ms/step - loss: 0.1763 - accuracy: 0.9203 - val_loss: 0.0654 - val_accuracy: 0.9780


In [130]:
y_pred=(model.predict(X_test) > 0.5).astype("int32")


In [131]:
# classes_x=model.predict>0.5.astype('int32')
#print(np.unique(X_test))
# print(y_pred)
print(accuracy_score(y_test,y_pred))

0.9873401855101529


### We must save our model so we can use it for making practical predictions. We must extract the preprocessing that we did for the training data so it's easy to input actual data so we will put all of that into a function

In [64]:
# Save the Model
# model.save('models/third_model')
# model.save('models/third_model_dropout_relu.h5')
# loaded_model = tf.keras.models.load_model('models/third_model_dropout_relu.h5')

INFO:tensorflow:Assets written to: models/third_model/assets


INFO:tensorflow:Assets written to: models/third_model/assets


In [23]:
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5000, 64)          640000    
                                                                 
 dropout (Dropout)           (None, 5000, 64)          0         
                                                                 
 lstm (LSTM)                 (None, 5000, 100)         66000     
                                                                 
 dropout_1 (Dropout)         (None, 5000, 100)         0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 1

In [132]:
def convert_article(article,trained_model=model):
    
    voc_size = 10000
    vector_length = 5000
    
    ## Clean the article
    stopW = stopwords.words('english')
    cleaned_1 = ' '.join([article])
    cleaned_2 = ' '.join([lemmatizer.lemmatize(word) for word in cleaned_1.split()])
    encoded = one_hot(cleaned_2,voc_size)
    embedded_doc = pad_sequences([encoded],padding='pre',maxlen=vector_length)
    #y_pred=(model.predict(X_test) > 0.5).astype("int32")
    return trained_model.predict([embedded_doc])# > 0.5).astype("int32")


In [133]:
fake_1 = all_data[all_data['type']==1].iloc[0:100]['text'].tolist()

### Now lets try Using this model with articles I've pulled through the internet
### The training dataset typically consisted of polictal and world news 
### For real data I've decided to use a variety of sources both that play to the bias of the dataset and other sources like tech,crypto,videogame, and sports aswell as satire news sources
### Becasue we thought the results could be interesting because the satire typically takes inspiration from the obsurdity of real events.
- The Onion
- The beaverton


In [169]:
real_world = {
'beaver' : f'''BATTLE CREEK, MI – Kellogg’s is moving to replace the 1,400 striking workers at several major US plants after contract negotiations broke down earlier this week. Hoping to stamp out any further dissension, Kellogg’s has also announced that several other employees who engaged in ‘acts of solidarity’ will also be replaced. Notable among them, Tony the Tiger of Frosted Flakes fame.

Despite his imminent dismissal, the cartoon spokestiger has only grown more outspoken on the topic of worker’s rights. Speaking at a rally on Friday, he addressed BCTGM union members.

“I can’t believe they’d treat their employees this poorly”, Tony said, his trademark red neckerchief defiantly covering his face. “These are same employees who worked g-r-rueling 80-hour work weeks to keep these plants open at the height of the pandemic. Would it kill them to show a little g-r-ratitiude?”

“This whole thing just stinks of corporate g-r-reed. Kellogg’s made $4 billion in profit this year and where does it go? Into the pockets of money-gr-r-rubbing CEOs. God, those gr-r-easy bastards gr-r-rind my gears to no end!”, he added to cheers from the crowd. 

Replacing Tony will be Kellogg’s new scab mascot Jerry the Jaguar. Jerry, a cartoon jaguar rendered in MS Paint by non-union artists and voiced by voice actors found on Fiverr. Jerry will soon be “spotted” on boxes of Frosted Flakes at grocery stores across the country sporting his signature green bandana and a tongue slightly too big for his mouth; reminding children everywhere that they’re not just good,  “they’re gr-r-fine.”

A relative unknown in the breakfast mascot industry; Jerry’s only previous work, according to his IMDB page, was representing a now-discontinued breakfast cereal called “Corn Shapes.”

“Look, Kellogg’s loves Tony and we wish him nothing but the best in his future endeavors,” said spokesperson Donna Winters at a publicity event for Jerry the Jaguar. “But his message of kindness, fair-play, and sportsmanship just doesn’t resonate with the Kellogg’s brand anymore. Besides, we think the kids are really going to respond well to Jerry and his message of ‘taking what you can get’ and ‘keeping your head down.”

At press time, a truly shocking amount of pornographic furry art featuring Jerry the Jaguar had been posted on the internet.''',

'cnn' : f'''Former White House trade adviser Peter Navarro has refused to comply with a Congressional subpoena for documents related to the Trump administration's response to the coronavirus, saying the former president ordered him not to, according to a letter Navarro wrote to the subcommittee investigating.
Navarro served as one of former President Donald Trump's pandemic response advisers and was, in part, responsible for curating the response to the virus. CNN previously reported he warned the White House in a January 2020 memo the virus could become a "full-blown pandemic."
The House Select Subcommittee on the Coronavirus Crisis issued the subpoena last month as part of its probe into the federal response to the pandemic, originally requesting Navarro provide documents by December 8 and a deposition on December 15.

Defying the request for documents Navarro wrote in a letter on December 7, "It is a direct order that I should not comply with the subpoena."

The subcommittee has still given Navarro until December 15 to sit for a deposition and demanded again Saturday he turn over relevant documents.

Navarro said in his letter to the subcommittee -- and addressed the letter to "Representative Rayburn" -- he would not cooperate because Trump told him to "protect executive privilege."
Navarro presumably intended to address the letter to Rep. James Clyburn, a Democrat and chairman of the coronavirus crisis panel. The subcommittee resides in the Rayburn House Office Building.

The Washington Post first reported on the refusal.
Clyburn responded in a letter to Navarro he had no valid basis for refusing to comply and he had waived any privilege by writing about conversations with Trump on the coronavirus response.
The refusal is "particularly indefensible given that you disclosed many details about your work in the White House, including details of conversations with the former President about the pandemic response, in your recent book and related press tour," Clyburn said in a letter to Navarro on Saturday.
Failing to comply with a subpoena can put a potential witness in contempt of Congress, which can lead to escalating financial penalties and the possibility of jail time.
One of Trump's former advisers, Steve Bannon, was indicted for contempt of Congress last month over his refusal to testify to the committee investigating the January 6 attack on the Capitol.
Navarro could not be immediately reached for a response to Clyburn's letter.

In a statement, Trump confirmed he asked Navarro not to cooperate.
"The Communist Democrats are engaging in yet another Witch Hunt, this time going after my Administration's unprecedented and incredible coronavirus response, despite the fact that, sadly, more Americans have died this year from Covid than in all of 2020," he said in a statement. "It is a Witch Hunt ... I'm telling Peter Navarro to protect executive privilege and not let these unhinged Democrats discredit our great accomplishments. The Witch Hunts must end!"''',
'cnn2' : f'''Connecticut man John Griffin was arrested Friday and charged with three counts of using a facility of interstate commerce to attempt to entice minors to engage in unlawful sexual activity, the United States Attorney's Office for the District of Vermont said in a news release.
Griffin, 44, has been a producer with CNN for about eight years.
"The charges against Mr. Griffin are deeply disturbing," a spokesperson for the network said in a statement Saturday. "We only learned of his arrest yesterday afternoon and have suspended him pending investigation."

The charges stem from conversations between Griffin and the purported parents of minor daughters, in which he allegedly tried to persuade them to "allow him to train their daughters to be sexually submissive," as well as an incident in which prosecutors allege unlawful sexual activity occurred with a 9-year-old girl, the news release said.

CNN has not been able to identify an attorney for Griffin.

According to an indictment filed on Thursday, Griffin allegedly paid for and kept an online profile on a "BDSM dating, Fetish, and Kink Site" for several years leading up to July 2020, on which he allegedly wrote he was looking for women who were "sexually 'submissive' and 'open-minded.'"
During conversations on separate online platforms, Griffin allegedly said he believed in "a way of life" in which "women are sexually subservient and inferior to men," according to the indictment. He allegedly used online platforms to seek parents "who would allow him to train their minor daughters to be sexually subservient," the indictment said.
Around February 2020, Griffin allegedly acquired a Vermont ski house in which he offered to host mothers and their minor daughters for "sexual training" purposes, according to the indictment.
Between April and June 2020, prosecutors allege Griffin talked online with the purported father of a 14-year-old girl and separately, the mother of a 16-year-old girl, proposing they begin "sexual training," the indictment said.
In June and July, Griffin also allegedly talked online and on the phone with someone who claimed to be the mother of two girls, 9 and 13, according to the indictment.
Griffin allegedly paid more than $3,000 to the woman to fly her and the 9-year-old girl to Boston in July, where prosecutors say he picked them up from the airport and brought them to the ski house "where the child was directed to engage in and did engage in illegal sexual activity," the indictment said.

"The allegations are deeply disturbing, and our office is committed to working with our partners at the United States Attorney's Office District of Vermont to ensure Mr. Griffin is held accountable for his actions," the FBI said in a statement posted on Twitter Friday. "The FBI, along with our law enforcement partners, will continue to aggressively investigate those who victimize the most vulnerable in our communities."
If convicted, Griffin faces anywhere from 10 years in prison to a life sentence for each count, according to the news release.''',
'gwpundit' : '''Florida – New jailhouse documents obtained by Fox News reveal 39-year-old Semmie Williams likely murdered 14-year-old Ryan Rogers because of anti-white racism.

Williams was arrested in Miami late last month after he fatally stabbed Rogers near an overpass on November 15.

The teen boy was stabbed “numerous times in the head and face,” according to the police report.

Palm Beach Gardens Police Chief initially said the murder was a “random act.
Newly released records show Williams used anti-white racial slurs after he was arrested and booked.

Williams also appeared to admit he murdered the boy because of what whites did to blacks in the past.

“While fingerprinting his left hand, I asked him if he understood the charges,” Officer Michael McCabe wrote in an incident report. “He stated, ‘Yea, murder, because of what they did to Black people about giving them syphilis.'”

Williams called one of the police officers “white devil” after he tried to break free from the officer.

Read the jailhouse records here:”''',

'onion':f'''WASHINGTON—In an effort to educate the opposition party on the true value of civic engagement, congressional Democrats staged an elaborate 4th of July pageant Friday in an effort to teach their Republican colleagues the importance of democracy.

Sources confirmed that the four-hour “star-spangled spectacular,” which involved months of preparation headed up by Senate Majority Leader Chuck Schumer, provided one blow-out night of freedom, fun, and refreshments. The Independence Day pageant, staged for the benefit of Congressional Republicans to promote consensus-building and working together, reportedly featured a variety of songs, dances, and skits from Democratic representatives tied to the loose theme of “America The Beautiful.”

“The Republicans refuse to accept the majesty and preciousness of our democracy, and so we have no choice but to offer them this no-holds-barred, musical extravaganza,” said Senate Majority Leader Chuck Schumer (D-NY), pointing to a scene in which Tim Kaine (D-VA) donned a dress and bonnet to portray Susan B. Anthony as well as members of the Oversight and Reform Committee filing into the auditorium playing “Yankee Doodle Dandy” on the recorder as particularly uplifting moments that helped convey the responsibility of honorably serving your constituents. “We’ve been rehearsing for months, learning choreography, and researching our state’s foods and traditions to really show how amazing this country can be, and our hope is that everybody who sees the show is going to come away feeling the weight of history on their shoulders, as well as humming a few classic patriotic tunes.”

“It is not hyperbole to say that our nation is on the precipice of disaster,” continued Schumer. “So, we’ve been pulling out all the stops to make sure that every performance really dazzles.”
Several eyewitness accounts revealed that the pageant featured involvement from every Democratic legislator except Joe Manchin (D-WV) who refused to participate if there were no Republican performers present. The patriotic program also included Sen. Ed Markey (D-MA) proudly marching in place while singing “My Country ’Tis Of Thee” and House Speaker Nancy Pelosi wearing a homemade Statue of Liberty costume and twirling on the stage with American flag handkerchiefs.

Attendees confirmed that the show concluded with a lively, heartfelt climax in which the entire caucus held up sparklers while singing a medley including “This Land Is Your Land” and “You’re A Grand Old Flag,” adding that despite a few minor flubs, such as Sen. Bernie Sanders (I-VT) struggling for several minutes to light a cigarette while playing FDR and Sen. Dianne Feinstein (D-CA) forgetting her line about California’s main exports and running offstage crying, it was a generally enjoyable event.

According to sources, several Democrats expressed disappointment that no Republican officials had chosen to attend apart from Sen. Rand Paul (R-KY), who reportedly only stayed long enough to fill a plate with red-white-and-blue sugar cookies before quickly exiting.

“I can’t believe they learned all those lines,” said Heather Fenton, mother of Sen. Jon Ossoff (D-GA), confirming that she had gotten some priceless footage of her son and his friends in their “absolutely adorable” outfits. “I’ll admit, the speaker system was pretty spotty so you couldn’t always hear what they were saying, and the A/C stopped working halfway through so I stepped out for about half an hour, but you could really see all the hard work they put into this, and I just couldn’t be more proud of them.” 
“I’m not sure anyone who wasn’t a friend or family member would have enjoyed it,” Fenton continued. “But it made them feel like they could make a difference, and that’s the most important thing.” ''',
'the_verge' : f'''Former Apple employee Cher Scarlett says the company didn’t adequately live up to its half of a settlement, so she won’t withdraw her complaint to the National Labor Relations Board as agreed, according to Forbes.

Scarlett says that her settlement with Apple required it to post “a company-wide memo clarifying employee rights including discussing pay & working conditions,” “in a prominent and visible location on the People site.” She says that the company did post a notice to its site, but that it was only up during the week that Apple employees were given off for Thanksgiving. “I’d argue all day that 7 days while no one is online for a holiday is absolutely not prominent and visible,” Scarlett tweeted on Thursday.

TheNLRB saysit can reject a withdrawal request made because of a private settlement if it “violates the National Labor Relations Act or Board policy.” Scarlett toldThe Verge that the board initially rejected her request, and sent a list of 22 changes that the board would need Apple to make before it would approve the withdrawal. According to Forbes, one of the requested changes removed wording that would’ve prevented Scarlett from encouraging someone “to file any charge or complaint with any administrative agency or Court against Apple” for a year. According to Scarlett, Apple refused to make those changes.

“I technically could ask for a unilateral withdrawal at this point,” Scarlett said in a message to The Verge, saying that she had been “interested in doing that to avoid witnesses having to give testimony, because realistically the memo is the best outcome we would have gotten from the board.” According to Scarlett, the group of potential witnesses decided it was worth it to continue with the complaint, potentially risking retaliation for testifying, given how Apple handled posting the memo.

Scarlett left the company in November, after filing the complaint that accused the company of engaging “in coercive and suppressive activity that has enabled abuse and harassment of organizers of protected concerted activity.” While at the company, she worked to let employees discuss and gather data about pay equity and was part of #AppleToo, a campaign about the company’s failure to deal with harassment and discrimination.''',
'onion2' : f'''EAST RUTHERFORD, NJ—Throwing the flag during the annual Army-Navy game after the clearing smoke revealed an obvious illegal hit, Army’s football team received a 15-yard penalty Saturday for drone-striking the kicker. “I don’t know what Army was thinking there—you’re not allowed to drone-strike a defenseless player,” commented CBS play-by-play announcer Brad Nessler, saying that hitting Navy’s kicker with a missile during a field goal attempt and evaporating his body was an easy penalty call for the officials. “You can see that Army’s coach, Jeff Monken, is getting frustrated with his drone operator on the sideline, and it’s understandable. You’ve got to save those drone strikes for ball carriers, and keep the drone strikes to a minimum on special teams. That’s going to give Navy good field position going into the next drive, so we’re going to have to see Army ramp up their M16s on defense if they want to keep this game from getting out of hand early on.” At press time, the referee was standing in the smoldering crater where the kicker’s body once was to overturn the drone-striking penalty after ruling that the offensive line had false-started on the play.''',
'news_punch2' : f'''Dr. Naomi Wolf, former Clinton admin advisor, has warned that the ‘New World Order’ are lying to keep the public enslaved so they can advance their agenda.

Speaking on Steve Bannon’s War Room, Dr. Wolf declared:

    We’re seeing this state by state.  There’s some kind of contract where governors have to deliver a certain percent of vaccinated in order to get something from pharma or in order to fullfill their contract.  So you can see this structure over and over of, well we have to reach 70% of 80% of vaccinated or you don’t get your rights back.  And, this is not how America works.  I have my rights.  She (New York Governor Hochul) can’t make me…

    …They’re trying to drag us onto their field of rhetoric, and it is a field of rhetoric of lies and it’s built on lies.  And I always think of Goebbels saying if you just tell a big enough lie over and over again people will believe you.  So number one, as I say all the time, everybody agrees, all the data show vaccinated, unvaccinated, that it does not affect transmission.  So all of these tyranical measures are nonsensical because they’re predicated on transmission.

    The other thing I want to say is “We’re not in a pandemic emergency anymore.  It’s not a pandemic.”''',
'news_punch' : f'''Dr. Anthony Fauci has said that a booster shot that specifically targets the Omicron covid variant may not be needed.

He did say however, that big pharma companies will still develop the variant specific shots.

Fauci told the health news website STAT: “I’m not so sure that we’re going to have to get a variant-specific boost vaccine to get an adequate protection from Omicron….because if you look at protection against variants, it appears to relate to the level of immunity and the breadth of the immunity that any given vaccine can instill on you.”

RT reports: While some experts have expressed concerns that the Omicron variant of the coronavirus is better able to evade the current crop of vaccines, Fauci has continued to place his faith in these shots. So too has Pfizer CEO Albert Bourla, though he has claimed that a fourth dose may be needed to combat the new strain.

“With Omicron we need to wait and see because we have very little information. We may need [the fourth dose] faster,” he told CNBC this week.

While Fauci said that an Omicron-specific booster likely won’t be needed, he did add that drug companies like Pfizer “are going to be making variant-specific boosters.”

Thankfully, the symptoms of the Omicron variant have been described by doctors and officials as mild, and as of Friday, the World Health Organization has not recorded any deaths as a result of the strain.''',
'news_punch3':f'''The man suing CNN anchor Don Lemon for violently sexually assaulting him has spoken out against the network, warning it is “a predator protecting machine” that must be shut down.

Dustin Hice, who filed a lawsuit against Lemon two years ago, accused the CNN host of violently sexually assaulting him at a New York bar during the summer of 2018.

The case is expected to go to court in 2022.

“They’re a network rife with predators and perverts,” Hice told Fox News.

“Lemon will have to testify under oath in the near future, but it’s good to see that there’s finally some accountability happening.”

Townhall.com reports: Hice explained that he is looking for closure and peace of mind after the incident with Lemon. He alleged that the encounter consisted of Lemon putting his hand down the front of his shorts, “vigorously” rubbing his genitalia and putting the same hand onto Hice’s face while asking a vulgar question.

CNN has yet to discipline Lemon, just like the network failed to do regarding Cuomo until this past week.

“This is who they are. They’re a predator-protecting machine, they slander and smear victims with impunity,” Hice said.  

Hice, who said he refused multiple settlement offers from CNN, believes the network is doing whatever it takes to protect Lemon from liability.

“They’ve tried to grind me down, they’ve attempted to intimidate witnesses in my case, they’ve released confidential information about me in attempt to doxx me. They are complicit. This is who they are,” Hice said.

Lemon’s attorney, Caroline J. Polisi, said Hice’s allegations against her client are “baseless” and “salacious.”

“In response to Mr. Hice’s baseless allegations, I encourage people to review the public filings, which show that Mr. Hice’s claims have been steadily eroding when they have been exposed to the civil litigation process,” Polisi told Fox News Digital. “Mr. Hice’s fanciful and salacious allegations against Mr. Lemon have collapsed of their own weight. Unlike Mr. Hice, Mr. Lemon has litigated, and will continue to litigate this case in the courtroom, not the press. We look forward to the approaching trial so he can finally put this case behind him.”

Hice’s lawsuit, filed in 2019 in Suffolk County Court, claims Lemon “intensely pushed his fingers against Plaintiff’s face under Plaintiff’s nose, forcing Plaintiff’s head thrust backward as Defendant repeatedly asked Plaintiff, ‘Do you like p—y or d–k?'”

The suit also alleges Lemon “continued to shove his fingers into Plaintiff’s face with aggression and hostility” throughout the interaction.

CNN suspended Cuomo Tuesday after the New York state attorney general released documents Monday showing he had used his media contacts to find information about women who accused his brother, former New York Gov. Andrew Cuomo (D), of sexual harassment and was in regular contact with the governor’s top aide about his findings. The network brought in an outside law firm to review the documents.

And on Wednesday, the network was informed that a former colleague of Cuomo’s at ABC News had accused him of sexual misconduct, an incident separate from an allegation from Shelly Ross, who in September wrote a guest essay in The New York Times accusing the anchor of touching her buttocks at a work party in 2005, when the two of them worked at ABC News.

Cuomo was terminated Saturday after “additional information,” not related to his involvement in his brother’s scandal, came to light.

“Chris Cuomo was suspended earlier this week pending further evaluation of new information that came to light about his involvement with his brother’s defense,” CNN said in a statement. “We retained a respected law firm to conduct the review, and have terminated him, effective immediately. While in the process of that review, additional information has come to light. Despite the termination, we will investigate as appropriate.”

Chris Cuomo released a statement of his own following his termination, saying that this “is not how I want my time at CNN to end but I have already told you why and how I helped my brother.”

“So let me now say as disappointing as this is, I could not be more proud of the team at Cuomo Prime Time and the work we did as CNN’s #1 show in the most competitive time slot,” he said. “I owe them all and will miss that group of special people who did really important work.”

Cuomo announced Monday that he will no longer be doing his SiriusXM radio show, Let’s Get After It.''',
'breitbart' : f'''

WASHINGTON, D.C. — A review of FBI Director James Comey’s professional history and relationships shows that the Obama cabinet leader — now under fire for his handling of the investigation of Hillary Clinton — is deeply entrenched in the big-money cronyism culture of Washington, D.C. His personal and professional relationships — all undisclosed as he announced the Bureau would not prosecute Clinton — reinforce bipartisan concerns that he may have politicized the criminal probe.

These concerns focus on millions of dollars that Comey accepted from a Clinton Foundation defense contractor, Comey’s former membership on a Clinton Foundation corporate partner’s board, and his surprising financial relationship with his brother Peter Comey, who works at the law firm that does the Clinton Foundation’s taxes.

Lockheed Martin

When President Obama nominated Comey to become FBI director in 2013, Comey promised the United States Senate that he would recuse himself on all cases involving former employers.

But Comey earned $6 million in one year alone from Lockheed Martin. Lockheed Martin became a Clinton Foundation donor that very year.

Comey served as deputy attorney general under John Ashcroft for two years of the Bush administration. When he left the Bush administration, he went directly to Lockheed Martin and became vice president, acting as a general counsel.

How much money did James Comey make from Lockheed Martin in his last year with the company, which he left in 2010? More than $6 million in compensation.

Lockheed Martin is a Clinton Foundation donor. The company admitted to becoming a Clinton Global Initiative member in 2010.

According to records, Lockheed Martin is also a member of the American Chamber of Commerce in Egypt, which paid Bill Clinton $250,000 to deliver a speech in 2010.

In 2010, Lockheed Martin won 17 approvals for private contracts from the Hillary Clinton State Department.

HSBC Holdings

In 2013, Comey became a board member, a director, and a Financial System Vulnerabilities Committee member of the London bank HSBC Holdings.

“Mr. Comey’s appointment will be for an initial three-year term which, subject to re-election by shareholders, will expire at the conclusion of the 2016 Annual General Meeting,” according to HSBC company records.

HSBC Holdings and its various philanthropic branches routinely partner with the Clinton Foundation. For instance, HSBC Holdings has partnered with Deutsche Bank through the Clinton Foundation to “retrofit 1,500 to 2,500 housing units, primarily in the low- to moderate-income sector” in “New York City.”

“Retrofitting” refers to a Green initiative to conserve energy in commercial housing units. Clinton Foundation records show that the Foundation projected “$1 billion in financing” for this Green initiative to conserve people’s energy in low-income housing units.

Who Is Peter Comey?

When our source called the Chinatown offices of D.C. law firm DLA Piper and asked for “Peter Comey,” a receptionist immediately put him through to Comey’s direct line. But Peter Comey is not featured on the DLA Piper website.

Peter Comey serves as “Senior Director of Real Estate Operations for the Americas” for DLA Piper. James Comey was not questioned about his relationship with Peter Comey in his confirmation hearing.

DLA Piper is the firm that performed the independent audit of the Clinton Foundation in November during Clinton-World’s first big push to put the email scandal behind them. DLA Piper’s employees taken as a whole represent a major Hillary Clinton 2016 campaign donation bloc and Clinton Foundation donation base.

DLA Piper ranks #5 on Hillary Clinton’s all-time career Top Contributors list, just ahead of Goldman Sachs.

And here is another thing: Peter Comey has a mortgage on his house that is owned by his brother James Comey, the FBI director.

Peter Comey’s financial records, obtained by Breitbart News, show that he bought a $950,000 house in Vienna, Virginia, in June 2008. He needed a $712,500 mortgage from First Savings Mortgage Corporation.

But on January 31, 2011, James Comey and his wife stepped in to become Private Party lenders. They granted a mortgage on the house for $711,000. Financial records suggest that Peter Comey took out two such mortgages from his brother that day.

This financial relationship between the Comey brothers began prior to James Comey’s nomination to become director of the FBI.

DLA Piper did not answer Breitbart News’ question as to whether James Comey and Peter Comey spoke at any point about this mortgage or anything else during the Clinton email investigation.

Peter Comey Re-Designed the FBI Building

FBI Director James Comey grew up in the New Jersey suburbs with his brother Peter. Both Comeys were briefly taken captive in 1977 by the “Ramsey rapist,” but the boys managed to escape through a window in their home, and neither boy was harmed.

James Comey became a prosecutor who worked on the Gambino crime family case. He went on to the Bush administration, a handful of private sector jobs, and then the Obama administration in 2013.

Peter Comey, meanwhile, went into construction.

After getting an MBA in real estate and urban development from George Washington University in 1998, Peter Comey became an executive at a company that re-designed George Washington University between 2004 and 2007 while his brother was in town working for the Bush administration.

In January 2009, at the beginning of the Obama administration, Peter Comey became “a real estate and construction consultant” for Procon Consulting.

Procon Consulting’s client list includes “FBI Headquarters Washington, DC.”

So what did Procon Consulting do for FBI Headquarters? Quite a bit, apparently. According to the firm’s records:

    Procon provided strategic project management for the consolidation of over 11,000 FBI personnel into one, high security, facility.

    Since 1972 the Federal Bureau of Investigation has had its headquarters in a purpose built 2.1 million square foot building on Pennsylvania Avenue. Having become functionally obsolete and in need of major repairs, GSA and the FBI were considering ways to meet the space needs required to maintain the Bureau’s mission and consolidate over 11,000 personnel.

    Procon assisted GSA in assessing the FBI’s space needs and options for fulfilling those needs. Services provided included project management related to site evaluations, budgeting, due diligence, and the development of procurement and funding strategies.

Those “funding strategies” included talking to “stakeholders”: “Worked with stakeholders and key leadership to identify strategic objectives, goals and long range plans for capital and real estate projects.”

Procon Consulting obtained its contract for FBI Headquarters prior to James Comey’s nomination to serve as director of the FBI.

In June 2011, Peter Comey left Procon Consulting to become “Senior Director of Real Estate Operations for the Americas” for DLA Piper.

Peter Comey has generated some controversy in that role. According to Law360 in May 2013 (the same month that James Comey was confirmed as someone being considered by Obama to become FBI director):

    Two real estate services businesses filed a $10 million suit against the law firm Monday alleging it stiffed them on as much as $760,000 of work done at DLA Piper’s Chicago office and improperly gave proprietary information to a competitor.

    ….

    The plaintiffs take particular aim at Peter Comey, DLA Piper’s senior director of real estate operations. Leasecorp and SpaceLogik include several emails in the complaint that are purportedly from DLA Piper senior real estate partners Jay Epstein and Rich Klawiter and are sharply critical of Comey’s handling of the matter. In one email, Epstein wrote that “it’s an embarrassment for the firm to be treating someone who we are working with like this.”

    In another email allegedly from Klawiter on Feb. 20, the DLA Piper partner informed Leasecorp President Michael Walker, a principal for both plaintiffs, that Comey had sent him and Epstein an email claiming that the real estate services firms were behind on their contractual obligations.

    “I just received an email from Peter (Jay was also a recipient) that is so inflammatory I can’t even send it or you’ll hit the roof,” Klawiter said in the email, according to the complaint. “This is not going to end well.”

''',
'greatgameindia' : f'''Last year a mysterious shipment was caught smuggling Coronavirus from Canada. It was traced to Chinese agents working at a Canadian lab. Subsequent investigation by GreatGameIndia linked the agents to Chinese Biological Warfare Program from where the virus is suspected to have leaked causing the Wuhan Coronavirus outbreak.

Note: BuzzFeed Reporter Who Attacked GreatGameIndia’s Coronavirus Bioweapon Story, Fired For Plagiarism

The findings of this investigation has been corroborated by none other than the Bioweapons expert Dr. Francis Boyle who drafted the Biological Weapons Convention Act followed by many nations. The report has caused a major international controversy and is suppressed actively by a section of mainstream media.

Infographic based on this report – The Secret History Of Coronavirus

Coronavirus Bioweapon
Coronavirus Bioweapon – How Chinese agents stole Coronavirus from Canada and weaponized it into a Bioweapon
The Saudi SARS Sample

On June 13, 2012 a 60-year-old Saudi man was admitted to a private hospital in Jeddah, Saudi Arabia, with a 7-day history of fever, cough, expectoration, and shortness of breath. He had no history of cardiopulmonary or renal disease, was receiving no long-term medications, and did not smoke.

Egyptian virologist Dr. Ali Mohamed Zaki isolated and identified a previously unknown coronavirus from his lungs. After routine diagnostics failed to identify the causative agent, Zaki contacted Ron Fouchier, a leading virologist at the Erasmus Medical Center (EMC) in Rotterdam, the Netherlands, for advice. 
Abnormalities on Chest Imaging of the Saudi patient infected with Coronavirus
Abnormalities on Chest Imaging of the Saudi patient infected with Coronavirus. Shown are chest radiographs of the patient on the day of admission (Panel A) and 2 days later (Panel B) and computed tomography (CT) 4 days after admission (Panel C).

Fouchier sequenced the virus from a sample sent by Zaki. Fouchier used a broad-spectrum “pan-coronavirus” real-time polymerase chain reaction (RT-PCR) method to test for distinguishing features of a number of known coronaviruses known to infect humans.''',
'tatum_reort' : f'''Evidence is mounting that the FBI was involved in planning and executing the January 6 Capitol event after a report dropped Monday night silently from Revolver News. The moment January 6 happened, the suspicion that this wasn’t coming from the MAGA movement was immediate.

Conservatives immediately saw this for what it was and refused to believe that MAGA people did anything but protest peacefully. Revolver says the evidence points to the FBI as a major contributing factor if not the culprit behind the January 6 psyop. 

The report tackled three questions about January 6 (1/6):

    “In the year leading up to 1/6 and during 1/6 itself, to what extent were the three primary militia groups (the Oath Keepers, the Proud Boys, and the Three Percenters) that the FBI, DOJ, Pentagon and network news have labeled most responsible for planning and executing a Capitol attack on 1/6 infiltrated by agencies of the federal government, or informants of said agencies?
    Exactly how many federal undercover agents or confidential informants were present at the Capitol or in the Capitol during the infamous “siege” and what roles did they play (merely passive informants or active instigators)?
    Finally, of all of the unindicted co-conspirators referenced in the charging documents of those indicted for crimes on 1/6, how many worked as a confidential informant or as an undercover operative for the federal government (FBI, Army Counterintelligence, etc.)?”

First, Revolver News broke the “siege” participants into two categories based on the level of crimes committed. The report gathered them into “Mostly Harmless Tourists” and “Unindicted Co-Conspirators Who Belonged To Any Of The Big Three’ Militia Groups.’ ”

The first category, “most harmless tourists,” encompasses most of those arrested after January 6. Julie Kelly published a gripping article that showcased letters from the political prisoners. Jonathan Mellis, son of a decorated Vietnam vet, was denied bail by the same judge that refused to drop the charges against Gen. Flynn.

Mellis wrote in part: 

“We are charged with every possible offense and held in the DC jail on solitary [sic] confinement and treated inhumanely. For example, a correctional officer from a different pod came to C2B screaming at us late at night on 6/1/21 because we had just sang ‘God Bless America’ [sic] from behind our locked doors like we do every night. Being as we are on lockdown 22 hours a day it’s nice to keep morale up through patriotism. When [name omitted by American Greatness], my next door [sic] neighbor, informed the officer that we were just singing ‘God Bless America’ the officer responded by yelling, ‘F*** [sic] America!’

I am concerned for the safety of myself and my fellow Capitol rioters here in the DC jail. We are locked down all day and threatened with violence regularly. We all know that getting our hands tied together and being beaten is something the DC jail officers have already done to Capitol rioters in this pod.

Solitary Confinement and beatings. That is our reality. When will the inhumane treatment end? I just want to let everyone know the reality of how we are treated in this place. Left wing [sic] rioters are not even held in jail. Much less subjected to the harsh and inhumane treatment my fellow Capitol rioters and I have survived under so far this year.”

 

According to the Revolver report, the second group, unindicted co-conspirators, “closely align” with the “violent insurrectionist” caricature that we hear about in the media and that the government claims to be targeting with “aggressive prosecutions.” The assertion here is that federal informants or undercover operatives infiltrated the militia groups and instigated the violence.

The report stated, “If it turns out that extraordinary percentage of the members of these groups involved in planning and executing the Capitol Siege were federal informants or undercover operatives, the implications would be nothing short of staggering.”

This would imply that the federal government knew about the attacks and did nothing to thwart them because the federal agencies, tasked with protecting us, “were active instigators in the most egregious and spectacular aspects of 1/6, amounting to a monumental entrapment scheme used as a pretext to imprison otherwise harmless protestors at the Capitol – and in a much larger sense used to frame the entire MAGA movement as potential domestic terrorists.”

The proof the FBI was actively using these practices “we need only go back a few months prior to 1/6 – to the so-called ‘Whitmer Kidnapping Plot.’ “Remember the kidnapping plot against Michigan Governor Whitmer? The question I had at the time was: “Why is the FBI able to track these men down organizing across state lines, and yet ANTIFA/BLM rioters are on video in multiple different states telling you they are going to burn cities to the ground, and they aren’t capable of finding the leader?”

The answer is now more clear. Revolver reports that we now know “of the 14 individuals who allegedly plotted the “kidnapping” and overthrow of the state government, at least five were undercover agents and federal informants.” The FBI even used the same milia group, the “Three Percenters,” for Gov. Whitmer and January 6.

And on top of that, “the director of the Detroit FBI Field Office, who oversaw the infiltration operation of the Michigan Plot, was subsequently granted a highly coincidental promotion to the DC office, where he is now the lead FBI agent for all 1/6 cases.”

On Tucker Carlson last night, Darren Beattie spoke about the Revolver News report, and a stunned Carlson asked him: ” After seeing all of this, you have to ask yourself: Does the national security apparatus do anything but conspire against the American people?”

The answer is a simple but firm no. It’s time to tear them down to the studs and bring them back under our control.''',
'notrealnews' : f'''As the world is gearing up for the coronavirus pandemic, cities are vowing to not let the virus move in to prevent the worst. During the Great Influenza Pandemic of 1918, World War I had already begun. Both events led to massive absenteeism, reducing production of pharmaceuticals for public health and sanitary purposes, and the need for screening for potential pandemic viruses, required by international treaties.

Public health is more fragile than ever today, and still heavily dependent on drug manufacturing capacity. Access to funding for pharmaceutical research and development, and basic tests, are both harder and harder.

Each year, the National Institutes of Health (NIH) grants millions to industries on the hopes of developing potential treatments, even though only about 15-20% of applications meet the criteria for granting a grant. The stakes are quite high in terms of life-saving products, but the uncertainty of funding can be paralyzing for many.

Five years ago, would it have been easy to imagine the world’s financial crisis in the context of pandemics. This time, however, the financial crisis was intertwined with the pandemic, rendering it even harder for hardtech and moonshots to get funding for product development or expansion.

Of course, modern technology makes pandemics a bit easier to forecast. Companies in the public health and pharmaceutical industry, faced with fewer options to invest in, have shifted from good old single-drug testing to mapping infectious diseases along the structural and molecular networks required to kill or control them with greater precision. New technology, and specifically, the targeted surveillance process used in collecting data to guide policy, may make our pandemics better.

As a researcher at Columbia University’s Center for the Study of Drug Development, I’ve experienced how hard it is to support new product development that is based around scalable or cheap manufacturing processes. The decrease in funding for the public health and pharmaceutical industries has further muted my colleagues’ enthusiasm for new platforms and technologies. The thought of spending millions to gain a competitive advantage over more ambitious competitors, despite the potential advantages, simply seems to be too risky.

After years of incubating, funding and working on multiple platforms, we have most of what we need. However, we can’t do it on our own. The public health and pharmaceutical industries have to figure out how to find global market-share for products that would benefit both industry and the rest of the world, including our food system.

The tenacity of the software industry, however, may provide some hope. Fundraising for hardtech, or hardtech-based moonshots, will be arduous. Cremades' favorite hardtech idea will always be consumer autonomy. Things that need to know the user to start, instead of the user having to do things that they don’t need.

Would funding of consumer autonomy be possible even when the world didn’t have a major pandemic? For sure, but there’s a limit to the amount of time you can spend solving problems that also concern large stakeholders (the public, the tech industry, and the industry itself). The global threat perception surrounding the pandemic has made funding harder than ever.

While finding product market fit after the coronavirus pandemic is going to be hard because of how huge the economic downturn is, I do see several opportunities.

Public engagement with disease research and surveillance may encourage the public to adopt new products and systems, in the same way it’s prompted other industries to get active in the innovation game.'''
}

In [171]:
for k,v in real_world.items():
    print(f'article:{k}, result:{convert_article(v)}')

article:beaver, result:[[0.00481387]]
article:cnn, result:[[0.00828139]]
article:cnn2, result:[[0.00362464]]
article:gwpundit, result:[[0.00228828]]
article:onion, result:[[0.02415777]]
article:the_verge, result:[[0.00774566]]
article:onion2, result:[[0.00540014]]
article:news_punch2, result:[[0.01684748]]
article:news_punch, result:[[0.00504156]]
article:news_punch3, result:[[0.0159876]]
article:breitbart, result:[[0.03910823]]
article:greatgameindia, result:[[0.00456028]]
article:tatum_reort, result:[[0.00888953]]
article:notrealnews, result:[[0.007694]]


In [70]:
for i in fake_1:
    test = convert_article(i)
    print(test)

[[0.9999925]]
[[0.99967957]]
[[0.999969]]
[[0.99999]]
[[0.99999416]]
[[0.999724]]
[[0.99999416]]
[[0.99999464]]
[[0.9997795]]
[[0.9998901]]
[[0.9999939]]
[[0.9999932]]
[[0.9999932]]
[[0.9999939]]
[[0.9999943]]
[[0.9999945]]
[[0.9999926]]
[[0.99999356]]
[[0.99999464]]
[[0.9999944]]
[[0.9999933]]
[[0.99999285]]
[[0.9999794]]
[[0.9999943]]
[[0.99999416]]
[[0.9999939]]
[[0.9999939]]
[[0.9999448]]
[[0.9999925]]
[[0.99999106]]
[[0.99999404]]
[[0.9999949]]
[[0.9999939]]
[[0.99978477]]
[[0.9999926]]
[[0.99999464]]
[[0.9999945]]
[[0.99999297]]
[[0.99999404]]
[[0.99988484]]
[[0.9999937]]
[[0.9999924]]
[[0.99999356]]
[[0.9999937]]
[[0.99999285]]
[[0.9999943]]
[[0.9999939]]
[[0.99999404]]
[[0.99999475]]
[[0.9999937]]
[[0.9999949]]
[[0.9999926]]
[[0.9999949]]
[[0.9999937]]
[[0.9999831]]
[[0.9999943]]
[[0.9999939]]
[[0.9999937]]
[[0.9999937]]
[[0.99999225]]
[[0.9999536]]
[[0.9999949]]
[[0.9999937]]
[[0.9999949]]
[[0.9999949]]
[[0.9999949]]
[[0.99999213]]
[[0.99997175]]
[[0.99999475]]
[[0.99998975]]


#### So... our model didnt do well against real world data we went out and find, it was able to classify real news well but not fake news we went out and found

In [137]:
new_dataset = pd.read_csv('./data/fake_or_real_news.csv')

In [139]:
real = new_dataset[new_dataset['label']=='REAL']
fake = new_dataset[new_dataset['label']=='FAKE']

In [141]:
fake_texts = fake.text.tolist()
real_texts = real.text.tolist()

In [162]:
real_results = [(convert_article(x)>0.5).astype("int32")  for x in real_texts[0:1000] ]
fake_results = [(convert_article(x) > 0.5).astype("int32")  for x in fake_texts[0:1000] ]


In [163]:

real['output'] = 0
fake['output'] = 1

fake_pred = fake.output.tolist()
real_pred = real.output.tolist()
real_results = np.concatenate(real_results,axis=1)[0]
fake_results = np.concatenate(fake_results,axis=1)[0]


In [172]:

#real_results
print(accuracy_score(fake_results,fake_pred[:1000]))
print(accuracy_score(real_results,real_pred[:1000]))
#print(accuracy_score(fake_results,fake_pred))

0.005
0.987
